Importing all the required libraries

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re

Reading the csv files

In [3]:
df_fake_path = '/Users/siddharth/Code/Python/FakeNewsDetection/data/Fake.csv'
df_true_path = '/Users/siddharth/Code/Python/FakeNewsDetection/data/True.csv'

df_fake = pd.read_csv(df_fake_path)
df_true = pd.read_csv(df_true_path)

In [4]:
df_fake.shape, df_true.shape

((23481, 4), (21417, 4))

Adding a column, 'reliability' to both dataframes to denote the credibility of the article (True - true news, False - fake news)

In [5]:
df_fake["reliability"] = False
df_true["reliability"] = True

Creating separate testing files, from origial dataframe, by choosing 10 random rows of data

In [6]:
df_fake_testing = df_fake.sample(10)
df_true_testing = df_true.sample(10)

df_fake_training = df_fake.drop(df_fake_testing.index)
df_true_training = df_true.drop(df_true_testing.index)

In [7]:
df_fake_training.shape, df_true_training.shape

((23471, 5), (21407, 5))

In [ ]:
df_manual_testing = pd.concat([df_fake_testing,df_true_testing], axis = 0)
manual_testing_path = '/Users/siddharth/Code/Python/FakeNewsDetection/data/manual_testing.csv'
df_manual_testing.to_csv(manual_testing_path)